In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

# Setting up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Upload data to S3
s3_bucket = "your-bucket-name"
s3_prefix = "crop-prediction"
train_data_s3_path = f"s3://{s3_bucket}/{s3_prefix}/train.csv"

# Define and Train Model on AWS SageMaker
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(entry_point="crop_pred.py",
                            framework_version="0.23-1",
                            role=role,
                            instance_type="ml.m5.large")

sklearn_estimator.fit({"train": train_data_s3_path})

# Deploy the model
predictor = sklearn_estimator.deploy(instance_type="ml.m5.large")




In [ ]:

import json
import boto3

s3_client = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker-runtime')

def lambda_handler(event, context):
    # Get data from event
    input_data = json.loads(event['body'])

    # Invoke SageMaker Endpoint for Prediction
    response = sagemaker_client.invoke_endpoint(
        EndpointName="crop-prediction-endpoint",
        ContentType="application/json",
        Body=json.dumps(input_data)
    )

    prediction = json.loads(response['Body'].read().decode())

    return {
        'statusCode': 200,
        'body': json.dumps({'prediction': prediction})
    }